## LimitFlux2core yeast example for parallel execution

Tyler W. H. Backman

Required inputs:
* input model
* exchange fluxes including biomass, and carbon feed uptake
* starting list of reactions which are considered 'core' reactions

Output produced:
* an updated (improved) set of core reactions

In [1]:
# run this to get libsbml for importing old models on a fresh launch of the debian-cheminformatics docker container

# import os
# os.system('pip3 install python-libsbml-experimental')

In [2]:
import sys
sys.path.append('../lftc')
import lftc

In [3]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:1,.5f}'.format # don't use scientific notation in output
import multiprocessing
import re
import cobra
print(cobra.__version__)

0.8.1


In [4]:
# import models
# model = cobra.io.read_sbml_model('test_data/yeast/iMM904TKs.xml')
# cobra.io.write_sbml_model(model, "test_data/yeast/iMM904TKs_sbml3.xml")
model = cobra.io.read_sbml_model('test_data/yeast/iMM904TKs_sbml3.xml')
model

Name,_mnt_nfs_jbei_nfs_users_hgmartin_transfer_iMM904
Memory address,0x07f09903845f8
Number of metabolites,1235
Number of reactions,1586
Objective expression,-1.0*biomass_SC5_notrace_reverse_e32ff + 1.0*biomass_SC5_notrace
Compartments,"Cytoplasm, Extracellular, Mitochondrion, Vacuole, Endoplasmic_reticulum, Peroxisome, Golgi, Nucleus"


In [5]:
# set exchange fluxes to some sample experimental values from 
# Ghosh 2016
# 13C Metabolic Flux Analysis for Systematic Metabolic Engineering of S. cerevisiae for Overproduction of Fatty Acids
# FLUXwyr1Etoh.txt

model.reactions.biomass_SC5_notrace.lower_bound = 0.02
model.reactions.biomass_SC5_notrace.upper_bound = 0.022
model.reactions.EX_etoh_LPAREN_e_RPAREN_.lower_bound = -1.18
model.reactions.EX_etoh_LPAREN_e_RPAREN_.upper_bound = -1.18
model.reactions.EX_glc_LPAREN_e_RPAREN_.lower_bound = 0
model.reactions.EX_glc_LPAREN_e_RPAREN_.upper_bound = 0
model.reactions.EX_ac_LPAREN_e_RPAREN_.lower_bound = -0.07
model.reactions.EX_ac_LPAREN_e_RPAREN_.upper_bound = -0.05
model.reactions.EX_ddca_LPAREN_e_RPAREN_.lower_bound = 0.004
model.reactions.EX_ddca_LPAREN_e_RPAREN_.upper_bound = 0.03
model.reactions.EX_ttdca_LPAREN_e_RPAREN_.lower_bound = 0.006
model.reactions.EX_ttdca_LPAREN_e_RPAREN_.upper_bound = 0.03
model.reactions.EX_hdca_LPAREN_e_RPAREN_.lower_bound = 0.02
model.reactions.EX_hdca_LPAREN_e_RPAREN_.upper_bound = 0.03
model.reactions.EX_ocdca_LPAREN_e_RPAREN_.lower_bound = 0.002
model.reactions.EX_ocdca_LPAREN_e_RPAREN_.upper_bound = 0.03
model.reactions.EX_lac_DASH_D_LPAREN_e_RPAREN_.lower_bound = 0
model.reactions.EX_lac_DASH_D_LPAREN_e_RPAREN_.upper_bound = 0

In [6]:
# set fatty acid production to a given fraction of max theoretical yield

productyield = 1.0

objective = {}
model.objective = objective
reaction = model.reactions.get_by_id('EX_ttdca_LPAREN_e_RPAREN_')
reaction.upper_bound = 500
objective[reaction.forward_variable] = 1
objective[reaction.reverse_variable] = -1
model.objective.set_linear_coefficients(objective)
model.objective.direction = 'max'
maxProduction = model.optimize()['EX_ttdca_LPAREN_e_RPAREN_']
reaction.lower_bound = productyield * maxProduction
reaction.upper_bound = reaction.lower_bound
print(reaction.upper_bound)

0.0186665


In [7]:
with open('test_data/yeast/REACTIONSwry1Glc.txt') as f:
    allLines = f.readlines()
reactionLines = list(filter(lambda l: re.match('^\w', l), allLines))
coreReactionNamesFromFile = set([re.sub('\s.*$', '', l) for l in reactionLines])
coreReactionNamesFromFile = set([re.sub('\(', '_LPAREN_', l) for l in coreReactionNamesFromFile])
coreReactionNamesFromFile = set([re.sub('\)', '_RPAREN_', l) for l in coreReactionNamesFromFile])
coreReactionNames = coreReactionNamesFromFile.intersection([r.id for r in model.reactions])

print(str(len(coreReactionNamesFromFile)) + ' core reactions in transitions file')
print(str(len(coreReactionNames)) + ' overlapping with model')
print('missing from model: ', coreReactionNamesFromFile.difference(coreReactionNames))

173 core reactions in transitions file
173 overlapping with model
missing from model:  set()


In [13]:
# run simulated annealing in parallel
def anneal(seed):
    auto_schedule = {'steps': 1000, 'tmax': 50000, 'tmin': 0.01, 'updates': 100}
    ocp = lftc.OptimalCoreProblem(
        state=coreReactionNames,
        model=model, 
        feed='EX_etoh_LPAREN_e_RPAREN_',
        minOverlapWithStart=1.0,
        excludeReactions=exchanges)
    ocp.set_schedule(auto_schedule)
    coreReactions, score = ocp.anneal(seed=seed)
    return [coreReactions, score]

exchanges = [r.id for r in model.exchanges]
cpuCores = 2
seeds = list(range(1,cpuCores*5,5))
pool = multiprocessing.Pool(processes=cpuCores)
results = pool.map(anneal, seeds)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
 Temperature        Energy    Accept   Improve     Elapsed   Remaining
  9163.96251          3.40   100.00%    10.00%     0:00:12     0:01:34

KeyboardInterrupt: 

In [ ]:
print('best new core:', min([r[1] for r in results]))

In [ ]:
exchanges = [r.id for r in model.exchanges]
ocp = lftc.OptimalCoreProblem(
    state=coreReactionNames,
    model=model, 
    feed='EX_etoh_LPAREN_e_RPAREN_',
    minOverlapWithStart=1.0,
    excludeReactions=exchanges)
print('old core:', ocp.energy())

In [ ]:
# print all non-zero producing reactions that feed into core
ocp.producingFluxes[ocp.producingFluxes > 0]

In [ ]:
ocp.consumingFluxes[ocp.consumingFluxes > 0]

In [ ]:
[r[1] for r in results]

In [ ]:
[len(r[0]) for r in results]

In [ ]:
bestNewCore = results[np.argmin([r[1] for r in results])][0]
len(bestNewCore)

In [ ]:
results[np.argmin([r[1] for r in results])][0].difference(coreReactionNames)

In [ ]:
# save results to file
import pickle
with open('results.p', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)